In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

In [2]:
model_trained = WhisperForConditionalGeneration.from_pretrained('/workspace/whisper/pretrain_base')
model_origin = WhisperForConditionalGeneration.from_pretrained('openai/whisper-base')
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="vi", task="transcribe")

In [6]:
input_speech, rate = librosa.load('/workspace/audio_test/vi_woman.wav', sr=16000)
input_features = processor(input_speech, sampling_rate=rate, return_tensors="pt").input_features

In [7]:
predicted_ids_model_trained = model_trained.generate(input_features, forced_decoder_ids=forced_decoder_ids)
predicted_ids_model_org = model_origin.generate(input_features, forced_decoder_ids=forced_decoder_ids)

In [8]:
processor.batch_decode(predicted_ids_model_trained), processor.batch_decode(predicted_ids_model_org), 

(['<|startoftranscript|><|vi|><|transcribe|><|notimestamps|>Theo sau quá trình tố tụng này, Hắc Khi M3 sẽ được xem xét lại bởi cơ quan lập pháp được bầu cử tiếp theo trong năm 2015 hoặc 2011 để tiếp tục quá trình xử lý.<|endoftext|>'],
 ['<|startoftranscript|><|vi|><|transcribe|><|notimestamps|> Theo Shao Quat trình tố tổng này, Hắc Chi Airaba sẽ được xem xét lại bởi cơ quan lập pháp được bầu cử tiếp theo trong năm 2015, hoặc 2016 để tiếp tục Quat trình sử lý.<|endoftext|>'])

## Evaluate model

In [2]:
from transformers import pipeline
from datasets import load_dataset, load_from_disk
from evaluate import evaluator
import evaluate
from tqdm import tqdm

In [3]:
metric = evaluate.load("wer")

In [4]:
transcriber = pipeline(
    "automatic-speech-recognition", 
    model="openai/whisper-base",
    generate_kwargs = {"language":"<|vi|>","task": "transcribe"}) # https://github.com/huggingface/transformers/issues/21809

In [5]:
# data_test = load_dataset("google/fleurs", "vi_vn", split="test", cache_dir='/workspace/whisper/dataset_hf_fleurs')
# data_test.save_to_disk('/workspace/whisper/dataset_hf_fleurs')
data_test = load_from_disk('/workspace/whisper/dataset_hf_fleurs')

In [13]:
preds = []
refs = []
for i in tqdm(range(len(data_test.select(range(0, 5))))):
    preds.append(transcriber(data_test[i]['audio']['array'])['text'])
    refs.append(data_test[i]['raw_transcription'])

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 5/5 [00:23<00:00,  4.77s/it]


In [15]:
preds

[' Văn hóa và bộ lạc cổ sưa đã bắt đầu dữ những công bật này để dễ lấy sữa tóc, thịt và da.',
 ' Nói My Spring Book, Tăng này đã giúp đời tiếng các thúc chối thuân 5 chẳng liền.',
 ' Nó cũng tấn công mọi thứ trong nước, ngày cả khủng lòng khẩn lò như tí vết cũng không phải là đối thủ với nó.',
 ' Xong người có mặt quá lớn nên việc mọi người tới được nơi tổ chức lễ tan tại Quảng Trường Thánh Bí Tơ là không thể.',
 ' Cho ó là lục địa tương nói nhỏ nhưng có nhiều quốc gia đục lập. Thông thường, việc đi qua nhiều quốc gia đồng nghỉa với việc phải xinh thị thức và sách, họ chứ nhiều lần.']

In [18]:
# metric = evaluate.load("wer")
# metric.compute(predictions=preds, references=refs)

0.5186400123494905

In [16]:
del transcriber
transcriber = pipeline(
    "automatic-speech-recognition", 
    model="/workspace/whisper/pretrain_base")

In [17]:
preds = []
refs = []
for i in tqdm(range(len(data_test.select(range(0, 5))))):
    preds.append(transcriber(data_test[i]['audio']['array'])['text'])
    refs.append(data_test[i]['raw_transcription'])

100%|██████████| 5/5 [00:38<00:00,  7.79s/it]


In [21]:
# metric = evaluate.load("wer")
# metric.compute(predictions=preds, references=refs)

0.35894566224143254

In [18]:
preds

['Văn hóa và bộ lạc cổ xưa đã bắt đầu giữ những con vật này để dễ lấy sữa tóc thực và gia.',
 'Đối với Sờ riêng Búc, trận này đã giúp đỡ tiếng các tốc chúi thua 5 chặng liền.',
 'Nó cũng tấn công mọi thứ trong nước, ngay cả khủng long khổng lồ như Tricic cũng không phải là đối thủ bên nó.',
 'Số người có mặt quá lớn nên việc mọi người tới được nơi tổ chức lễ tang tại Quảng Trường Thánh B tơ là không thể.',
 'Cho là lục địa tương đối nhỏ nhưng có nhiều quốc gia độc lập, thông thường việc đi qua nhiều quốc gia đồng nghĩa với việc phải xin thị thực và xét hoàn chiếu nhiều lần.']

## Sample prediction

Origin pred | Finetune pred | References |
|--|--|--|
|Văn hóa và bộ lạc cổ sưa đã bắt đầu dữ những công bật này để dễ lấy sữa tóc, thịt và da.|Văn hóa và bộ lạc cổ xưa đã bắt đầu giữ những con vật này để dễ lấy sữa tóc thực và gia.|Văn hóa và bộ lạc cổ xưa đã bắt đầu giữ những con vật này để dễ lấy sữa, tóc, thịt, và da.|
|Nói My Spring Book, Tăng này đã giúp đời tiếng các thúc chối thuân 5 chẳng liền.|Đối với Sờ riêng Búc, trận này đã giúp đỡ tiếng các tốc chúi thua 5 chặng liền.|Đối với Springboks, trận này đã giúp đội tuyển kết thúc chuỗi thua 5 trận liền.|
|Nó cũng tấn công mọi thứ trong nước, ngày cả khủng lòng khẩn lò như tí vết cũng không phải là đối thủ với nó.|Nó cũng tấn công mọi thứ trong nước, ngay cả khủng long khổng lồ như Tricic cũng không phải là đối thủ bên nó.|Nó cũng tấn công mọi thứ trong nước; ngay cả khủng long khổng lồ như T. rex cũng không phải là đối thủ với nó.|
|Xong người có mặt quá lớn nên việc mọi người tới được nơi tổ chức lễ tan tại Quảng Trường Thánh Bí Tơ là không thể.|Số người có mặt quá lớn nên việc mọi người tới được nơi tổ chức lễ tang tại Quảng Trường Thánh B tơ là không thể.|Số người có mặt quá lớn nên việc mọi người tới được nơi tổ chức lễ tang tại Quảng trường Thánh Peter là không thể.|
|Cho ó là lục địa tương nói nhỏ nhưng có nhiều quốc gia đục lập. Thông thường, việc đi qua nhiều quốc gia đồng nghỉa với việc phải xinh thị thức và sách, họ chứ nhiều lần.|Cho là lục địa tương đối nhỏ nhưng có nhiều quốc gia độc lập, thông thường việc đi qua nhiều quốc gia đồng nghĩa với việc phải xin thị thực và xét hoàn chiếu nhiều lần.|Châu Âu là lục địa tương đối nhỏ nhưng có nhiều quốc gia độc lập. Thông thường, việc đi qua nhiều quốc gia đồng nghĩa với việc phải xin thị thực và xét hộ chiếu nhiều lần.|